# Tutorial: Data class

In [14]:
import sys
from datetime import datetime
import numpy as np
import dask
import xarray as xr

In [15]:
# connect to dask client
from dask.distributed import Client

client = Client("tcp://127.0.0.1:49686")
client

<Client: 'tcp://127.0.0.1:49686' processes=4 threads=8, memory=16.00 GiB>

In [6]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%autoreload 2
sys.path.insert(0, '../')
from clouddrift.data import erddap, local

# Remote ERDDAP server [(link)](https://data.pmel.noaa.gov/generic/erddap/tabledap/gdp_hourly_velocities.html)

## Unique drifter

In [5]:
# get drifter from errdap
e = erddap()
e.retrieve_drifter(10050130)
e.print_constraints()

All variables in this dataset:
None

Constraints of this dataset:
{
 "ID=": "10050130"
}


In [6]:
%%time
ds = e.to_xarray()
ds

CPU times: user 64 ms, sys: 13.9 ms, total: 77.8 ms
Wall time: 1.26 s


<xarray.Dataset>
Dimensions:                (trajectory: 1, obs: 3592)
Coordinates:
    longitude              (obs) float32 ...
    latitude               (obs) float32 ...
    time                   (obs) datetime64[ns] ...
Dimensions without coordinates: trajectory, obs
Data variables: (12/47)
    ID                     (trajectory) object ...
    rowSize                (trajectory) int32 ...
    location_type          (obs) object ...
    WMO                    (obs) float64 ...
    expno                  (obs) float64 ...
    deploy_date            (obs) datetime64[ns] ...
    ...                     ...
    DrogueBallast          (obs) object ...
    DragAreaAboveDrogue    (obs) object ...
    DragAreaOfDrogue       (obs) object ...
    DragAreaRatio          (obs) object ...
    DrogueCenterDepth      (obs) object ...
    DrogueDetectSensor     (obs) object ...
Attributes: (12/51)
    acknowledgement:            Elipot et al. (2016). Global Drifter Program ...
    cdm_data_type:              Trajectory
    cdm_trajectory_variables:   ID
    comment:                    Global Drifter Program hourly data
    contributor_name:           NOAA Global Drifter Program
    contributor_role:           Data Acquisition Center
    ...                         ...
    subsetVariables:            ID, location_type, WMO, expno
    summary:                    Global Drifter Program hourly data
    time_coverage_end:          2012-07-14T18:00:00Z
    time_coverage_start:        2012-02-15T14:00:00Z
    title:                      Global Drifter Program hourly drifting buoy c...
    Westernmost_Easting:        103.261

## list of drifters

In [7]:
# get drifter from errdap
e = erddap()

drifter_ids = [10050130, 10051120]

dfs = {}
for i in drifter_ids:
    e.retrieve_drifter(i)
    dfs[i] = e.to_xarray()

In [8]:
dfs[10050130]

<xarray.Dataset>
Dimensions:                (trajectory: 1, obs: 3592)
Coordinates:
    longitude              (obs) float32 ...
    latitude               (obs) float32 ...
    time                   (obs) datetime64[ns] ...
Dimensions without coordinates: trajectory, obs
Data variables: (12/47)
    ID                     (trajectory) object ...
    rowSize                (trajectory) int32 ...
    location_type          (obs) object ...
    WMO                    (obs) float64 ...
    expno                  (obs) float64 ...
    deploy_date            (obs) datetime64[ns] ...
    ...                     ...
    DrogueBallast          (obs) object ...
    DragAreaAboveDrogue    (obs) object ...
    DragAreaOfDrogue       (obs) object ...
    DragAreaRatio          (obs) object ...
    DrogueCenterDepth      (obs) object ...
    DrogueDetectSensor     (obs) object ...
Attributes: (12/51)
    acknowledgement:            Elipot et al. (2016). Global Drifter Program ...
    cdm_data_type:              Trajectory
    cdm_trajectory_variables:   ID
    comment:                    Global Drifter Program hourly data
    contributor_name:           NOAA Global Drifter Program
    contributor_role:           Data Acquisition Center
    ...                         ...
    subsetVariables:            ID, location_type, WMO, expno
    summary:                    Global Drifter Program hourly data
    time_coverage_end:          2012-07-14T18:00:00Z
    time_coverage_start:        2012-02-15T14:00:00Z
    title:                      Global Drifter Program hourly drifting buoy c...
    Westernmost_Easting:        103.261

In [9]:
dfs[10051120]

<xarray.Dataset>
Dimensions:                (trajectory: 1, obs: 4269)
Coordinates:
    longitude              (obs) float32 ...
    latitude               (obs) float32 ...
    time                   (obs) datetime64[ns] ...
Dimensions without coordinates: trajectory, obs
Data variables: (12/47)
    ID                     (trajectory) object ...
    rowSize                (trajectory) int32 ...
    location_type          (obs) object ...
    WMO                    (obs) float64 ...
    expno                  (obs) float64 ...
    deploy_date            (obs) datetime64[ns] ...
    ...                     ...
    DrogueBallast          (obs) object ...
    DragAreaAboveDrogue    (obs) object ...
    DragAreaOfDrogue       (obs) object ...
    DragAreaRatio          (obs) object ...
    DrogueCenterDepth      (obs) object ...
    DrogueDetectSensor     (obs) object ...
Attributes: (12/51)
    acknowledgement:            Elipot et al. (2016). Global Drifter Program ...
    cdm_data_type:              Trajectory
    cdm_trajectory_variables:   ID
    comment:                    Global Drifter Program hourly data
    contributor_name:           NOAA Global Drifter Program
    contributor_role:           Data Acquisition Center
    ...                         ...
    subsetVariables:            ID, location_type, WMO, expno
    summary:                    Global Drifter Program hourly data
    time_coverage_end:          2012-05-31T06:00:00Z
    time_coverage_start:        2011-12-05T10:00:00Z
    title:                      Global Drifter Program hourly drifting buoy c...
    Westernmost_Easting:        110.08

## region

In [10]:
e2 = erddap()

lon = [-98, -78]
lat = [18, 31]
day0 = "%sZ" % datetime(2015,1,1).isoformat()
day1 = "%sZ" % datetime(2020,12,31).isoformat()
time = [day0, day1]

e2.retrieve_region(lon, lat, time)

e2.print_constraints()

All variables in this dataset:
None

Constraints of this dataset:
{
 "longitude>=": -98,
 "longitude<=": -78,
 "latitude>=": 18,
 "latitude<=": 31,
 "time>=": "2015-01-01T00:00:00Z",
 "time<=": "2020-12-31T00:00:00Z"
}


In [11]:
%%time
ds2 = e2.to_xarray()
ds2

CPU times: user 566 ms, sys: 129 ms, total: 695 ms
Wall time: 49.4 s


<xarray.Dataset>
Dimensions:                (trajectory: 202, obs: 316142)
Coordinates:
    longitude              (obs) float32 ...
    latitude               (obs) float32 ...
    time                   (obs) datetime64[ns] ...
Dimensions without coordinates: trajectory, obs
Data variables: (12/47)
    ID                     (trajectory) object ...
    rowSize                (trajectory) int32 ...
    location_type          (obs) object ...
    WMO                    (obs) float64 ...
    expno                  (obs) float64 ...
    deploy_date            (obs) datetime64[ns] ...
    ...                     ...
    DrogueBallast          (obs) object ...
    DragAreaAboveDrogue    (obs) object ...
    DragAreaOfDrogue       (obs) object ...
    DragAreaRatio          (obs) object ...
    DrogueCenterDepth      (obs) object ...
    DrogueDetectSensor     (obs) object ...
Attributes: (12/51)
    acknowledgement:            Elipot et al. (2016). Global Drifter Program ...
    cdm_data_type:              Trajectory
    cdm_trajectory_variables:   ID
    comment:                    Global Drifter Program hourly data
    contributor_name:           NOAA Global Drifter Program
    contributor_role:           Data Acquisition Center
    ...                         ...
    subsetVariables:            ID, location_type, WMO, expno
    summary:                    Global Drifter Program hourly data
    time_coverage_end:          2020-03-31T23:00:00Z
    time_coverage_start:        2015-01-01T00:00:00Z
    title:                      Global Drifter Program hourly drifting buoy c...
    Westernmost_Easting:        -97.76723

# Local

In [8]:
path_clouddrift = '../data/process/gdp_v2.00_obs.nc'
path_traj_clouddrift = '../data/process/gdp_v2.00_traj.nc'

In [9]:
%%time
l = local(path_clouddrift, path_traj_clouddrift)

CPU times: user 689 ms, sys: 29.6 ms, total: 719 ms
Wall time: 738 ms


In [10]:
l.ds_obs.ve

<xarray.DataArray 've' (obs: 165754333)>
dask.array<open_dataset-f06d454207b673da07b26b8b639bddcfve, shape=(165754333,), dtype=float32, chunksize=(66417,), chunktype=numpy.ndarray>
Coordinates:
    longitude  (obs) float32 dask.array<chunksize=(417,), meta=np.ndarray>
    latitude   (obs) float32 dask.array<chunksize=(417,), meta=np.ndarray>
    time       (obs) datetime64[ns] dask.array<chunksize=(417,), meta=np.ndarray>
    ids        (obs) uint64 dask.array<chunksize=(417,), meta=np.ndarray>
Dimensions without coordinates: obs
Attributes:
    long_name:  Eastward velocity
    units:      m/s

In [71]:
%%time
l.ds_obs.ve.mean().compute()

CPU times: user 1.15 s, sys: 30.4 ms, total: 1.18 s
Wall time: 7.68 s


<xarray.DataArray 've' ()>
array(0.00698143, dtype=float32)

## unique drifter

In [84]:
%%time
ds = l.retrieve_drifter(101509)

CPU times: user 48.3 ms, sys: 3.41 ms, total: 51.8 ms
Wall time: 74.5 ms


In [85]:
ds

<xarray.Dataset>
Dimensions:    (obs: 1970)
Coordinates:
    longitude  (obs) float32 136.3 136.3 136.3 136.3 ... 124.7 124.7 124.7 124.7
    latitude   (obs) float32 10.02 10.02 10.03 10.03 ... 13.71 13.71 13.72 13.72
    time       (obs) datetime64[ns] 2011-12-15T23:00:00 ... 2012-03-07
    ids        (obs) uint64 101509 101509 101509 101509 ... 101509 101509 101509
Dimensions without coordinates: obs
Data variables: (12/16)
    ve         (obs) float32 -0.1005 -0.0956 -0.1227 ... -0.1854 -0.1878 -0.1732
    vn         (obs) float32 0.0952 0.0941 0.1313 ... 0.0856 -0.0085 -0.0036
    gap        (obs) float32 4.579e+03 2.333e+03 ... 1.132e+04 1.901e+03
    err_lat    (obs) float32 0.00053 0.00107 0.00262 ... 0.00573 0.00276 0.00152
    err_lon    (obs) float32 0.00066 0.00505 0.00692 ... 0.00243 0.00367 0.00475
    err_ve     (obs) float32 0.0148 0.0623 0.0532 ... 0.0205 0.0216 0.0303
    ...         ...
    err_sst    (obs) float32 0.012 0.012 0.016 0.019 ... 0.014 0.013 0.014 nan
    err_sst1   (obs) float32 0.037 0.022 0.014 0.01 ... 0.016 0.022 0.033 nan
    err_sst2   (obs) float32 0.035 0.027 0.023 0.021 ... 0.022 0.028 0.033 nan
    flg_sst    (obs) uint8 5 5 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5 5 0
    flg_sst1   (obs) uint8 5 5 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5 5 0
    flg_sst2   (obs) uint8 5 5 5 5 5 5 5 2 2 2 2 2 5 ... 2 5 5 5 5 5 5 5 5 5 5 0
Attributes: (12/15)
    title:             Global Drifter Program hourly drifting buoy collection
    history:           Version 2.00.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-12-21T21:09:36.319435
    publisher_name:    GDP Drifter DAC
    publisher_email:   aoml.dftr@noaa.gov
    ...                ...
    metadata_link:     https://www.aoml.noaa.gov/phod/dac/dirall.html
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2022) to be submitted. Elipot et al. (2...
    summary:           Global Drifter Program hourly data

## list of drifters

In [89]:
%%time
ds = l.retrieve_drifter([101509, 9927907])

CPU times: user 1.23 s, sys: 1.94 s, total: 3.17 s
Wall time: 3.25 s


In [90]:
ds

<xarray.Dataset>
Dimensions:    (obs: 6542)
Coordinates:
    longitude  (obs) float32 136.3 136.3 136.3 136.3 ... -154.2 -154.2 -154.2
    latitude   (obs) float32 10.02 10.02 10.03 10.03 ... -51.89 -51.89 -51.88
    time       (obs) datetime64[ns] 2011-12-15T23:00:00 ... 1999-09-22T11:00:00
    ids        (obs) uint64 101509 101509 101509 ... 9927907 9927907 9927907
Dimensions without coordinates: obs
Data variables: (12/16)
    ve         (obs) float32 -0.1005 -0.0956 -0.1227 ... 0.063 0.0766 0.0786
    vn         (obs) float32 0.0952 0.0941 0.1313 ... 0.2224 0.2002 0.1971
    gap        (obs) float32 4.579e+03 2.333e+03 ... 9.504e+03 9.504e+03
    err_lat    (obs) float32 0.00053 0.00107 0.00262 ... 0.0039 0.00517 0.00692
    err_lon    (obs) float32 0.00066 0.00505 0.00692 ... 0.00394 0.00681 0.01003
    err_ve     (obs) float32 0.0148 0.0623 0.0532 ... 0.0684 0.0667 0.0651
    ...         ...
    err_sst    (obs) float32 0.012 0.012 0.016 0.019 0.018 ... nan nan nan nan
    err_sst1   (obs) float32 0.037 0.022 0.014 0.01 0.008 ... nan nan nan nan
    err_sst2   (obs) float32 0.035 0.027 0.023 0.021 0.018 ... nan nan nan nan
    flg_sst    (obs) uint8 5 5 5 5 5 5 5 5 5 5 5 5 5 ... 0 0 0 0 0 0 0 0 0 0 0 0
    flg_sst1   (obs) uint8 5 5 5 5 5 5 5 5 5 5 5 5 5 ... 0 0 0 0 0 0 0 0 0 0 0 0
    flg_sst2   (obs) uint8 5 5 5 5 5 5 5 2 2 2 2 2 5 ... 0 0 0 0 0 0 0 0 0 0 0 0
Attributes: (12/15)
    title:             Global Drifter Program hourly drifting buoy collection
    history:           Version 2.00.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-12-21T21:09:36.319435
    publisher_name:    GDP Drifter DAC
    publisher_email:   aoml.dftr@noaa.gov
    ...                ...
    metadata_link:     https://www.aoml.noaa.gov/phod/dac/dirall.html
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2022) to be submitted. Elipot et al. (2...
    summary:           Global Drifter Program hourly data

In [91]:
%%time
# retrieve 50 random drifters
idx_to_retrieve = l.ds_traj['ID'][np.random.randint(0, l.number_traj, 50)].values
ds = l.retrieve_drifter(idx_to_retrieve)

CPU times: user 2.35 s, sys: 1.99 s, total: 4.34 s
Wall time: 4.71 s


In [93]:
ds

<xarray.Dataset>
Dimensions:    (obs: 452907)
Coordinates:
    longitude  (obs) float32 -164.5 -164.5 -164.5 -164.5 ... 121.6 121.6 121.6
    latitude   (obs) float32 21.28 21.27 21.27 21.27 ... 22.03 22.04 22.04 22.04
    time       (obs) datetime64[ns] 2013-01-18T14:00:00 ... 2006-02-01T14:00:00
    ids        (obs) uint64 116090 116090 116090 116090 ... 56654 56654 56654
Dimensions without coordinates: obs
Data variables: (12/16)
    ve         (obs) float32 -0.096 -0.0933 -0.1451 ... -0.2385 -0.2427 -0.2496
    vn         (obs) float32 -0.1456 -0.1462 -0.1287 ... 0.1081 0.1041 0.0703
    gap        (obs) float32 6.048e+03 6.048e+03 ... 5.27e+03 6.134e+03
    err_lat    (obs) float32 0.00093 0.0006 0.0009 ... 0.00417 0.00297 0.00797
    err_lon    (obs) float32 0.00119 0.00267 0.00167 ... 0.00308 0.00139 0.00413
    err_ve     (obs) float32 0.0314 0.0347 0.0227 ... 0.0504 0.0376 0.0452
    ...         ...
    err_sst    (obs) float32 0.022 0.014 0.014 0.014 ... 0.02 0.02 0.021 nan
    err_sst1   (obs) float32 0.052 0.034 0.022 0.016 ... 0.024 0.037 0.064 nan
    err_sst2   (obs) float32 0.042 0.036 0.029 0.022 ... 0.034 0.045 0.062 nan
    flg_sst    (obs) uint8 5 5 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5 5 0
    flg_sst1   (obs) uint8 5 5 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5 5 0
    flg_sst2   (obs) uint8 2 2 2 5 5 5 2 2 2 2 5 5 5 ... 5 5 5 5 5 5 5 5 5 2 2 0
Attributes: (12/15)
    title:             Global Drifter Program hourly drifting buoy collection
    history:           Version 2.00.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-12-21T21:09:36.319435
    publisher_name:    GDP Drifter DAC
    publisher_email:   aoml.dftr@noaa.gov
    ...                ...
    metadata_link:     https://www.aoml.noaa.gov/phod/dac/dirall.html
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2022) to be submitted. Elipot et al. (2...
    summary:           Global Drifter Program hourly data

## region

In [94]:
%%time

lon = [-98, -78]
lat = [18, 31]
day0 = "%sZ" % datetime(2015,1,1).isoformat()
day1 = "%sZ" % datetime(2020,12,31).isoformat()
time = [day0, day1]
ds2 = l.retrieve_region(lon, lat, time)
ds2

distributed.client - WARNING - Couldn't gather 17324 keys, rescheduling {"('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 2369)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 11262)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 15008)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 3356)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 16824)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 7085)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 5077)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 10533)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 9741)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 10762)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 15860)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 6892)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 4718)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 11220)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 710)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 9461)": (), "('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 8565)": (), "('ge-6eb4b27c165bf1

CancelledError: ('ge-6eb4b27c165bf1c6761e7c7a98613ca9', 1784)

# Testing

## groupby and map()

In [28]:
%%time
#grouped_traj = l.ds_obs.groupby('ids')

# used smaller group for now
grouped_traj = ds2.groupby('ids')

CPU times: user 50.4 ms, sys: 1.32 ms, total: 51.7 ms
Wall time: 52.4 ms


In [71]:
grouped_traj

DatasetGroupBy, grouped over 'ids'
202 groups with labels 36953, 36956, ..., 68055200.

In [124]:
keys = list(grouped_traj.groups.keys())

In [128]:
grouped_traj[keys[150]]

<xarray.Dataset>
Dimensions:    (obs: 1220)
Coordinates:
    longitude  (obs) float32 -82.07 -82.1 -82.13 -82.16 ... -80.28 -80.29 -80.29
    latitude   (obs) float32 18.01 18.01 18.02 18.03 ... 25.25 25.25 25.25 25.25
    time       (obs) datetime64[ns] 2019-04-07T15:00:00 ... 2019-05-28T10:00:00
    ids        (obs) uint32 64017250 64017250 64017250 ... 64017250 64017250
Dimensions without coordinates: obs
Data variables:
    ve         (obs) float32 -0.9008 -0.9142 -0.9053 ... -0.1231 -0.1108 -0.1077
    vn         (obs) float32 0.2454 0.2727 0.2947 ... -0.0868 -0.0828 -0.0779
    err_lat    (obs) float32 50.0 37.0 37.0 50.0 38.0 ... 37.0 37.0 56.0 80.0
    err_lon    (obs) float32 53.0 39.0 39.0 53.0 39.0 ... 41.0 41.0 62.0 88.0
    err_ve     (obs) float32 0.0185 0.0137 0.0137 0.0184 ... 0.0137 0.0184 0.025
    err_vn     (obs) float32 0.0185 0.0137 0.0137 0.0184 ... 0.0137 0.0184 0.025
    gap        (obs) timedelta64[ns] 00:00:00 01:00:29 ... 00:00:00 01:00:29
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-10T18:08:56.738618
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

In [74]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    # distance
    earthRadius = 6371 # km
    d = 2 * np.arcsin(np.sqrt(a)) * earthRadius # km
    return d

In [79]:
def distance(ds):
    return haversine(ds.longitude[:-1], ds.latitude[:-1], 
                     ds.longitude[1:], ds.latitude[1:])

In [137]:
grouped_traj

DatasetGroupBy, grouped over 'ids'
202 groups with labels 36953, 36956, ..., 68055200.

In [138]:
grouped_traj.mean()

<xarray.Dataset>
Dimensions:  (ids: 202)
Coordinates:
  * ids      (ids) uint64 36953 36956 36963 36964 ... 67876760 68054710 68055200
Data variables:
    ve       (ids) float32 0.05257 0.1594 0.2091 ... 0.07468 0.007762 0.136
    vn       (ids) float32 -0.1262 -0.2204 -0.04286 ... 0.01196 0.8668 1.0
    err_lat  (ids) float32 0.001784 0.00238 0.002618 ... 41.41 41.19 41.2
    err_lon  (ids) float32 0.002222 0.003155 0.003806 ... 47.34 46.97 47.53
    err_ve   (ids) float32 0.03571 0.06082 0.06542 ... 0.0153 0.006513 0.006082
    err_vn   (ids) float32 0.02773 0.0439 0.04486 ... 0.0153 0.006513 0.006082
    gap      (ids) timedelta64[ns] 02:06:44.540000 ... 01:52:21.812030075

In [129]:
distance(ds2)

<xarray.DataArray (obs: 316164)>
array([3.3122504 , 3.648436  , 3.3347745 , ..., 0.42627728, 0.77198535,
       0.77314365], dtype=float32)
Dimensions without coordinates: obs

In [133]:
grouped_traj.map(distance)

DatasetGroupBy, grouped over 'ids'
202 groups with labels 36953, 36956, ..., 68055200.

In [102]:
def mean(obj, dim):

    # note: apply always moves core dimensions to the end

    return xr.apply_ufunc(

        np.mean, obj, input_core_dims=[[dim]], kwargs={"axis": -1}

    )

In [107]:
ds2['ve']

<xarray.DataArray 've' (obs: 316165)>
array([ 0.9168,  0.9245,  0.9377, ..., -0.0741, -0.0812, -0.0882],
      dtype=float32)
Coordinates:
    longitude  (obs) float32 -87.51 -87.48 -87.44 ... -78.84 -78.84 -78.85
    latitude   (obs) float32 29.42 29.42 29.41 29.4 ... 30.98 30.98 30.99 30.99
    time       (obs) datetime64[ns] 2016-01-23T15:00:00 ... 2019-12-21T17:00:00
    ids        (obs) uint32 102539 102539 102539 ... 68055200 68055200 68055200
Dimensions without coordinates: obs
Attributes:
    long_name:  Eastward velocity
    units:      m/s

In [108]:
mean(ds2['ve'], ds2.dims['obs'])

ValueError: operand to apply_ufunc has required core dimensions [316165], but some of these dimensions are absent on an input variable: [316165]

In [55]:
def mean_traj(ds):
    da = xr.DataArray(
    data=temperature,

    dims=["x", "y", "time"],

    coords=dict(

        lon=(["x", "y"], lon),

        lat=(["x", "y"], lat),

        time=time,

        reference_time=reference_time,

    ),

    attrs=dict(

        description="Ambient temperature.",

        units="degC",

    ),

)

In [57]:
grouped_traj

DatasetGroupBy, grouped over 'ids'
202 groups with labels 36953, 36956, ..., 68055200.

In [56]:
grouped_traj.apply(mean_traj)

<xarray.DataArray 'latitude' (ids: 202)>
array([28.555435, 27.344658, 24.699554, 28.671339, 26.457586, 24.419994,
       28.960884, 24.697721, 24.316198, 24.888361, 27.902721, 24.795607,
       25.06476 , 28.911047, 24.478098, 27.854866, 25.21522 , 24.920883,
       28.568882, 28.938877, 28.682173, 27.7735  , 26.441175, 29.197353,
       29.190804, 28.982203, 22.986967, 25.862604, 23.001366, 22.996271,
       22.998642, 28.231674, 22.995237, 24.679054, 22.99084 , 25.962938,
       29.029032, 27.733503, 28.775698, 28.821138, 26.471987, 27.147133,
       25.711628, 22.988634, 22.162504, 27.575388, 28.023384, 29.432585,
       20.370787, 25.236382, 24.778227, 18.153254, 20.865189, 21.33908 ,
       26.13048 , 21.49348 , 22.581371, 20.580751, 20.309761, 24.753605,
       27.983393, 30.562372, 27.896568, 30.06785 , 26.78535 , 25.14972 ,
       24.15759 , 25.131872, 25.723417, 28.166388, 21.987411, 23.596104,
       18.660065, 25.295666, 19.386936, 23.61912 , 25.909355, 20.84834 ,
       21.999554, 19.371923, 19.363586, 23.257483, 28.674164, 27.117514,
       22.278332, 20.912193, 28.994164, 29.703787, 19.349543, 29.876472,
       29.389818, 30.847708, 25.735205, 19.672403, 30.653652, 30.152521,
       30.366179, 22.909235, 20.393347, 21.231268, 21.165598, 22.021463,
       19.199099, 23.569374, 27.222118, 28.093578, 30.104805, 27.416698,
       28.072126, 26.546371, 27.434109, 25.31026 , 30.576595, 30.508156,
       22.888607, 23.652533, 22.830343, 24.590244, 27.917343, 18.052176,
       22.777271, 30.794893, 30.488907, 30.432396, 30.432243, 25.267618,
       24.7156  , 25.155184, 29.836094, 28.234331, 28.860292, 29.714752,
       30.9996  , 24.724373, 26.613115, 28.553188, 28.88467 , 30.555822,
       21.421707, 29.648615, 24.130041, 29.49835 , 20.125671, 28.134787,
       29.583677, 29.94218 , 30.115051, 28.311014, 29.96982 , 24.629448,
       23.635115, 26.089664, 29.820454, 18.326948, 29.76823 , 26.23564 ,
       21.226835, 28.897442, 22.307493, 25.36987 , 29.364597, 26.24374 ,
       26.124569, 25.473991, 24.319094, 25.725855, 29.044586, 25.3327  ,
       24.904839, 25.275686, 29.543123, 27.51553 , 25.950254, 28.616873,
       28.61285 , 25.30727 , 26.324335, 30.616798, 18.500984, 28.37214 ,
       27.051043, 29.261423, 30.632643, 28.631226, 30.60188 , 25.751526,
       28.623608, 30.67501 , 30.639263, 29.253952, 27.078142, 28.489363,
       26.851728, 29.579575, 28.769476, 28.015945, 25.808838, 28.956814,
       28.969307, 29.102648, 28.128399, 29.558378], dtype=float32)
Coordinates:
  * ids      (ids) uint64 36953 36956 36963 36964 ... 67876760 68054710 68055200

In [22]:
value = np.zeros(l.number_traj)

for i in range(0, len(l.traj_idx)-1):
    r = l.traj_idx[i:i+1]
    value[i] = l.ds_obs['longitude'][r].mean()

KeyboardInterrupt: 

In [17]:
%%time
mean_traj = grouped_traj.mean('obs')

KeyboardInterrupt: 

In [17]:
def midpoint_trajectory():
    return ds.longitude.mean()

In [26]:
%%time
grouped_traj_longitude = l.ds_obs['longitude'].groupby('ids')

CPU times: user 24.6 s, sys: 1.8 s, total: 26.4 s
Wall time: 27.3 s


In [29]:
grouped_traj_longitude

DataArrayGroupBy, grouped over 'ids'
17322 groups with labels 0, 2578, ..., 68318210, 68604340.

In [27]:
grouped_traj_longitude[0]

<xarray.DataArray 'longitude' (obs: 17482)>
dask.array<getitem, shape=(17482,), dtype=float32, chunksize=(7073,), chunktype=numpy.ndarray>
Coordinates:
    longitude  (obs) float32 dask.array<chunksize=(7073,), meta=np.ndarray>
    latitude   (obs) float32 dask.array<chunksize=(7073,), meta=np.ndarray>
    time       (obs) datetime64[ns] dask.array<chunksize=(7073,), meta=np.ndarray>
    ids        (obs) uint32 dask.array<chunksize=(7073,), meta=np.ndarray>
Dimensions without coordinates: obs
Attributes:
    long_name:  Longitude
    units:      degrees_east

In [28]:
%%time
grouped_traj_longitude.mean()

KeyboardInterrupt: 

In [20]:
%%time
grouped_traj.mean('longitude')

ValueError: cannot reduce over dimensions ['longitude']. expected either '...' to reduce over all dimensions or one or more of Frozen({'obs': 17482}).

In [97]:
%%time
grouped_traj = l.ds_obs[["ve", "vn"]].groupby('ids')

KeyboardInterrupt: 

In [24]:
grouped_traj

DatasetGroupBy, grouped over 'ids'
17322 groups with labels 0, 2578, ..., 68318210, 68604340.

In [38]:
grouped_traj[list(grouped_traj.groups.keys())[150]]

<xarray.Dataset>
Dimensions:    (obs: 7754)
Coordinates:
    longitude  (obs) float32 dask.array<chunksize=(7754,), meta=np.ndarray>
    latitude   (obs) float32 dask.array<chunksize=(7754,), meta=np.ndarray>
    time       (obs) datetime64[ns] dask.array<chunksize=(7754,), meta=np.ndarray>
    ids        (obs) uint32 dask.array<chunksize=(7754,), meta=np.ndarray>
Dimensions without coordinates: obs
Data variables:
    ve         (obs) float32 dask.array<chunksize=(7754,), meta=np.ndarray>
    vn         (obs) float32 dask.array<chunksize=(7754,), meta=np.ndarray>
Attributes: (12/17)
    title:             Global Drifter Program hourly drifting buoy collection
    id:                Global Drifter Program ID 13555
    history:           Version 1.04.  Metadata from dirall.dat and deplog.dat
    Conventions:       CF-1.6
    date_created:      2021-11-10T18:08:56.738618
    publisher_name:    GDP Drifter DAC
    ...                ...
    contributor_name:  NOAA Global Drifter Program
    contributor_role:  Data Acquisition Center
    institution:       NOAA Atlantic Oceanographic and Meteorological Laboratory
    acknowledgement:   Elipot et al. (2016). Global Drifter Program quality-c...
    doi:               10.1002/2016JC011716TBA
    summary:           Global Drifter Program hourly data

In [ ]:
%%time
grouped_traj.map(lambda ve: ve - ve.mean(), shortcut=False)

In [ ]:
grouped_traj[0].isel(traj=[100])

In [ ]:
grouped_traj[1]

In [ ]:
grouped_traj[100].isel(traj=[100])

# manually apply function at each trajectory

In [21]:
def distance(i):
    # get first and last point
    ds_subset = l.ds.isel(traj=[i], obs=slice(l.traj_idx[i], l.traj_idx[i + 1]))
    lon1 = ds_subset.longitude[0]
    lat1 = ds_subset.latitude[0]
    lon2 = ds_subset.longitude[-1]
    lat2 = ds_subset.latitude[-1]
    
    # convert to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [ ]:
%%time
a = distance(10)

In [32]:
a.compute()

KeyboardInterrupt: 

In [ ]:
import dask.bag as db
b = db.from_sequence(range(10), npartitions=10).map(distance)

In [ ]:
b

In [ ]:
%%time
d = b.compute()

In [ ]:
d

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(d)